# 서포트 벡터 머신
* 매우 강력하고 선형, 비선형 분류, 회귀, 이상치 탐색 등에 사용 가능한 다목적 머신러닝 모델
* 특히 복잡한 분류 문제에 잘 들어맞으며 작거나 중간 크기의 데이터셋에 적합하다.
---
### 선형 SVM분류
* 가장 폭이 넓은 도로를 찾는 것으로 생각하는 라지 마진 분류라고한다.
* 도로 바깥쪽에 훈련 샘플을 더 추가해도 결정 경계에는 영향을 미치지 않는다.
* 결정경계에 위치한 샘플에 전적으로 의지한다. 이를 서포트 벡터라고 한다.
---
### 소프트 마진 분류
* 모든 샘플이 도로 바깥쪽에 분류되면 이를 하드 마진 분류라고 한다.
* 이는 선형적으로 구분 될 수 있어야 제대로 작동하며 이상치에 민감하다는 단점을 가진다.
* 마진 오류 사이에 적절한 균형을 잡아야 하며, 이를 소프트 마진 분류라고 한다.
* 사이킷 런에서 이는 C라는 파라미터로 설정한다.
* 과대적합이라면 C를 감소시켜 모델을 규제할 수 있다.
* LinearSVC는 규제에 편향을 포함시킨다. 따라서 훈련 세트에서 평균을 빼서 중앙에 맞춰야한다.

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

iris=datasets.load_iris()
x=iris["data"][:,(2,3)]
y=(iris.target==2).astype(np.float64)

svm_clf=Pipeline([
                  ("scaler",StandardScaler()),
                  ("linear_svc",LinearSVC(C=1,loss="hinge"))
])

svm_clf.fit(x,y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [2]:
svm_clf.predict([[5.5,1.7]])

array([1.])

In [5]:
svc_clf=SVC(kernel="linear",C=1)# SVC모델 사용
svc_clf.fit(x,y)
svc_clf.predict([[5.5,1.7]])

array([1.])

### 비선형 SVM 분류
* 선형적으로 분류할 수 없는 데이터 셋이 많다.


In [12]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

x,y=make_moons(n_samples=100,noise=0.15)
polynomial_svm_clf=Pipeline([
                             ("poly_features",PolynomialFeatures(degree=3)),
                             ("scaler",StandardScaler()),
                             ("svm_clf",LinearSVC(C=10,loss="hinge"))
])
polynomial_svm_clf.fit(x,y)
polynomial_svm_clf.predict([[1.1,2.1]])

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([0])

### 다항식 커널
* 낮은 차수의 다항식은 복잡한 데이터셋을 표현하지 못한고 높으면 오래걸린다.
* 커널트릭이라는 수학적 기교를 적용 가능하다.
* 이는 특성을 추가하지 않으면서 추가한 것과 같은 결과를 얻을 수 있다.
* 이 코드는 3차 다항식 커널을 사용해 SVM 분류기를 훈련시킨다.
* coef0는 모델이 높은 차수와 낮은 차수에 얼마나 영향 받을지를 조절.

In [18]:
from sklearn.svm import SVC
poly_kernel_svm_clf=Pipeline([
                              ("sclaer",StandardScaler()),
                              ("svc_clf",SVC(kernel="poly",degree=3,coef0=1,C=5))
])
poly_kernel_svm_clf.fit(x,y)
poly_kernel_svm_clf.predict([[2.2,1.1]])

array([1])

### 유사도 특성
* 비선형 특성을 다루는 다른 기법은 랜드마크와 얼마나 닮았는지를 정하는 유사도 함수로 계산한 특성을 추가하는 것.
* 방사 기저 함수를 유사도 함수로 정의한다.
* 함수의 값은 0부터 1까지 변화하며 종모양으로 나타난다.
---
### 가우시안 RBF커널
* gamma를 증가시키면 종 모양 그래프가 좁아져서 각 샘플의 영향범위가 줄어든다. 따라서 결정경계가 불규칙해지고 구불구불 휜다.
* 감마가 작아지면 부드러워진다.
---
* 커널중 일단은 선형커널을 먼저 시도해본다. 특히 훈련 세트가 크거나 특성이 많으면 그러하다.
* 너무 안크다면 가우시안 RBF커널도 시도해보면 좋다.


In [22]:
rbf_kernel_svm_clf=Pipeline([
                             ("scaler",StandardScaler()),
                             ("svm_clf",SVC(kernel="rbf",gamma=5,C=0.001))
])
rbf_kernel_svm_clf.fit(x,y)
rbf_kernel_svm_clf.predict([[2.2,1.1]])

array([1])

### 계산 복잡도
* LinearSVC의 파이썬 계산 복잡도는 거의 선형적이다 대략 O(m*n)
* 정밀도를 노이면 수행시간이 길어진다. 이는 허용오차 파라미터 E(tol)로 조절한다.
* SVC커널의 트릭 알고리즘은 보통 O(M^2*n) 과 O(M^3*n) 사이이다.
* 샘플 수가 커지면 매우매우 느려진다.
---
### SVM 회귀
* 회귀로 사용하는 방법은 목표를 반대로 하는 것이다.
* 제한된 마진 오류 안에서 마진 안에 최대한 많은 샘플이 들어가도록 학습한다.
* 도로의 폭은 E(espilon)로 제한한다.
* 마진 안에서는 훈련 샘플이 추가되어도 모델의 예측에는 영향이 없다. 따라서 이 모델을 E에 민감하지 않다고 말한다.

In [37]:
from sklearn.svm import LinearSVR
svm_reg=LinearSVR(epsilon=0.5)
svm_reg.fit(x,y)
svm_reg.predict([[0.1,0.5]])

array([0.5])

In [36]:
from sklearn.svm import SVR
svm_poly_reg=SVR(kernel="poly",degree=2,C=100,epsilon=0.1)
svm_poly_reg.fit(x,y)
svm_poly_reg.predict([[0.1,0.5]])

array([0.58602708])

### SVM이론
* 